In [115]:
from sklearn import set_config
from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    RandomizedSearchCV,
    train_test_split,
)
from sklearn.pipeline import Pipeline
from scipy.stats import loguniform

from skfolio import RatioMeasure, RiskMeasure
from skfolio.datasets import load_factors_dataset, load_sp500_dataset
from skfolio.model_selection import (
    CombinatorialPurgedCV,
    WalkForward,
    cross_val_predict,
)
from skfolio.moments import (
    DenoiseCovariance,
    DetoneCovariance,
    EWMu,
    GerberCovariance,
    ShrunkMu,
)
from skfolio.optimization import (
    MeanRisk,
    NestedClustersOptimization,
    ObjectiveFunction,
    RiskBudgeting,
)
from skfolio.pre_selection import SelectKExtremes # type: ignore
from skfolio.preprocessing import prices_to_returns
from skfolio.prior import BlackLitterman, EmpiricalPrior, FactorModel
from skfolio.uncertainty_set import BootstrapMuUncertaintySet

In [116]:
import pandas as pd

### Loading out asset classes from asset_classes.csv

In [117]:
asset_classes = pd.read_csv('../data/asset_classes.csv',  index_col=0)

In [118]:
asset_classes.head()

,AGGG,FXI,HYG,S&P
Date,,,,
2024-10-11,4.40,33.37,79.57,5815.03
2024-10-10,4.40,33.22,79.36,5780.05
2024-10-09,4.40,32.79,79.40,5792.04
2024-10-08,4.41,33.33,79.45,5751.13
2024-10-07,4.41,36.69,79.26,5695.94


train/test split

In [119]:
X = prices_to_returns(asset_classes)
X_train, X_test = train_test_split(X, test_size=0.33, shuffle=False)

Specifying model.

This does some form of convex optimizartion

In [120]:
model = MeanRisk(
     objective_function=ObjectiveFunction.MINIMIZE_RISK,
     risk_measure=RiskMeasure.CVAR,
     risk_aversion= 0,
     budget=1,
     #max_short=0.2,
     risk_free_rate=0.04044 # Norges Bank has values on zero coupon yields
)

In [121]:
model.fit(X_train)

print(model.weights_)

[5.90025666e-01 1.01429018e-09 4.09974330e-01 3.00828124e-09]


In [122]:
portfolio = model.predict(X_test)

In [123]:
portfolio.composition

,weight
asset,
AGGG,0.590026
HYG,0.409974


In [124]:
print(portfolio.annualized_sharpe_ratio)
print(portfolio.summary())

-0.6826664945251603
Mean                                            -0.0047%
Annualized Mean                                   -1.19%
Variance                                         0.0023%
Annualized Variance                                0.59%
Semi-Variance                                   0.00099%
Annualized Semi-Variance                           0.25%
Standard Deviation                                 0.48%
Annualized Standard Deviation                      7.66%
Semi-Deviation                                     0.31%
Annualized Semi-Deviation                          4.99%
Mean Absolute Deviation                            0.27%
CVaR at 95%                                        1.09%
EVaR at 95%                                        1.94%
Worst Realization                                  2.86%
CDaR at 95%                                       11.91%
MAX Drawdown                                      17.27%
Average Drawdown                                   3.47%
EDaR at 95%

In [125]:
portfolio.composition

,weight
asset,
AGGG,0.590026
HYG,0.409974


In [126]:
portfolio.plot_returns()